In [ ]:
import os
import time
import ipaddress
import struct
from IPy import IP
from uuid import getnode as get_mac
from subprocess import Popen, PIPE
import re
import socket
import csv
import subprocess
import sys

In [ ]:
print("MAN IN THE MIDDLE ATTACK DETECTION TOOL")

In [ ]:
def get_arp_table(IP):
    cmd = "arp -a"
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=None, shell=True)
    output = process.communicate()
    interfaceBlocks = output[0].decode().split('Interface:')

    for interfaceBlock in interfaceBlocks:
        lines = interfaceBlock.split('\r\n');
        interfaceIP = lines[0].split('---')[0].strip()

        if interfaceIP == IP:
            names = ['Internet Address', 'Physical Address', 'Type']
            reader = csv.DictReader(
                lines[1:], fieldnames=names, skipinitialspace=True, delimiter=' ')
            next(reader)

            return [block for block in reader]

In [ ]:
# Get IP Address
def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
     # doesn't even have to be reachable
     s.connect(('10.255.255.255', 1))
     IP = s.getsockname()[0]
    except:
     IP = '127.0.0.1'
    finally:
     s.close()
    return IP

In [ ]:
IP = get_ip()
print("IP of victim machine is:",IP)
arp_table=get_arp_table(IP)
mac_address=arp_table[0]['Physical Address']
mitm = False
print("Gateway's MAC Address is:", mac_address, "\r\n")
print("Looking if there is MITM attack possible....")
for line in arp_table[1:]:
    if line['Physical Address'] == mac_address:
        mitm = True
if mitm:
    print("MITM Attack is Detected")
else:
    print("No MITM Attack Right Now")